In [1]:
import numpy as np
from itertools import *
import functools as fu
from numpy import kron
from numpy.linalg import cholesky, eig
import torch
import pandas as pd


from qiskit.quantum_info import random_density_matrix
from qutip import rand_dm_hs, Qobj, fidelity


c:\Users\Adriano.Palmieri\AppData\Local\Continuum\anaconda3\envs\qiskit\lib\site-packages\qutip\__init__.py:59: UserWarning: Old version of Cython detected: needed 0.29.20, got 0.29.15.
  warnings.warn(


### leading dimension to set

In [2]:
# number of particles
d = 4
#local dimension (qubit, qutrit...)
local_dim = 2 

 ## 1) depolarization and dephasing channel

here we provide a local and a global depolarizing channel, to introduce an extra physical channel. This files can be used in step 3 during the data generatoin and linear inversion pre-processing.

In [ ]:
def local_depolarizing_channel(qubits_number, density_matrix, probability):

		"""
		NOTE: this function must be used inside a for loop to get to apply each pauli's operator for each state

		"""

		pX = np.array([[0.,1.], [1.,0.]]) # X Pauli matrix
		pY = np.array([[0.,-1.j], [1.j, 0.]]) # Y Pauli matrix
		pZ = np.array([[1., 0.], [0.,-1.]]) # Z Pauli matrix
		pI = np.array([[1.,0.], [0.,1.]])

		for i in range(qubits_number):
			
			operator_string_x = list(repeat(pI,qubits_number))
			operator_string_y = list(repeat(pI,qubits_number))
			operator_string_z = list(repeat(pI,qubits_number))
			

			operator_string_x[i] = pX
			operator_string_y[i] = pY
			operator_string_z[i] = pZ
	
			X = fu.reduce(np.kron,operator_string_x)
			Y = fu.reduce(np.kron,operator_string_y)
			Z = fu.reduce(np.kron,operator_string_z)
		
			new_density_matrix = (1-probability)*density_matrix 
			
			new_density_matrix = new_density_matrix + (probability/3)*X.dot(density_matrix).dot(X)
			#print(new_density_matrix.shape)
			new_density_matrix = new_density_matrix + (probability/3)*Y.dot(density_matrix).dot(Y)

			new_density_matrix = new_density_matrix + (probability/3)*Z.dot(density_matrix).dot(Z)
				
		return new_density_matrix




In [ ]:
def global_depo_channel(dm,p):
    
    dim = dm.shape[0]
    I = np.eye(dim,dim)
    
    return (1-p)*dm + (p/(dim))*I
    

## Support functions:
1. the vectorization of the Cholesky matrices $vec(C_{ij})$.
2. positive semidefinite brute-force approximation of non physical matrices $\rho_{LI}$(eigenvaluescheck, PureEigenvaluesCheck)


In [3]:

def vectorization_new(rho):
	
	dim = rho.shape[0]
	#chol = np.linalg.cholesky(rho)
	chol = 	torch.linalg.cholesky(torch.complex(torch.Tensor(rho.real),torch.Tensor(rho.imag)))

	diag = np.diag(chol).real.tolist()

	lower_t_indeces = np.tril_indices_from(chol,k=-1)

	reals = list(chol[lower_t_indeces].real.flatten())
	imags = list(chol[lower_t_indeces].imag.flatten())

	return np.array(diag+reals+imags, dtype=np.float64)


def from_mpc_tonumpy(colored_rho):

	"""
	input: a density matrix, mpc type
	output: nd.array of the same density matrix recasted in complex128 type
	"""
	placeholder = np.zeros((colored_rho.shape[0],colored_rho.shape[1]),dtype=complex)
	for r in range(colored_rho.shape[0]):
		for c in range(colored_rho.shape[1]):
			a =np.double(mp.nstr(mp.re(colored_rho[r][c])))
			b = np.double(mp.nstr(mp.im(colored_rho[r][c])))
			built = complex(a,b)
			placeholder[r][c] = built
	return placeholder

def eigenvaluesCheck(dm):

	eis, eigvecs = eig(dm)

	eis[eis.real <0 ] = 0.0001
	eis = [ el.real for el in eis]
	eis = np.array(eis)

	cleanRho = eigvecs@ np.diag(eis) @ eigvecs.T.conj()

	return cleanRho/np.trace(cleanRho)
	
def PureEigenvaluesCheck(dm):

	eis, eigvecs = eig(dm)

	eis[eis.real.round(2) < 0.99 ] = 0.0001
	eis = [ el.real for el in eis]
	eis = np.array(eis)

	cleanRho = eigvecs@ np.diag(eis) @ eigvecs.T.conj()

	return cleanRho/np.trace(cleanRho)

## 2) Uploading the n qubits basis

here we upload the informationally complete set of basis, one for generate the Bhorn values $p_i= Tr(\rho\pi_i)$, and the corrispondenting dual $\tilde{\pi}_i$ to reconstruct our density matrix from experimental values $\rho_{back} =\sum_i  f_i\tilde{\pi}_i $

In [38]:
general_basis = np.load("basis-generation/4-qubits/4qubitsPauli.npy ", allow_pickle=True)
reconstruction_basis = np.load("basis-generation/4-qubits/4qubitsPauliReconstruction.npy", allow_pickle=True)

In [39]:
reconstruction_basis.shape

(256, 16, 16)

## 3) Random states generation, pre-processing, training dataset preparation.

Here we orderly do:

1. generate random matrices
2. calculate Born values
3. approximate with fine statistics, fixing the parameter "trials" (for SICS), or normalizing the Gaussian noise (for Pauli)
4. using linear inversion to reconstruct the density matrix, and approximate it to the close positive definite approximation thereof, killing the negative eigenvalues
5. vectorize the original matrix and the new, experimental one. This is our training dataset

In [ ]:
import os

totarr =[]
hs = []
tot =1
i = 0

trials = 1000

folder = '../multinomialData/'
filename = 'Haar4qubitsTrials'+str(trials)+'Sic.npy'

if not os.path.isdir(folder):
    os.mkdir(folder)

def HSdist(A,B):
    return np.trace((A-B)@(A-B)).real
  
hsm = []
while True:

	#rho_start0 = np.array(rand_dm_hs(local_dim**d).full())
	#PURE STATES
	rho_startA = np.array(random_density_matrix(local_dim**d,rank=1, method = "Hilbert-Schmidt").data)
	#rho_start0 = global_depo_channel(rho_start0,0.1)
	s_borns =  np.array([ np.trace(rho_startA @ base).real for base in general_basis])
	
	#MULTINOMIAL. This is for Guillem,PRA alike tests!!
	s_borns_approx = np.random.multinomial(trials, s_borns)/trials
	
	#reconstruct the approximation, first step of linear inversion
	
	rback = sum([ np.round(reconstruction_basis[i],12)*s_borns_approx[i] for i in range(4**d) ])
	#brute force approximation upon LI
	cleanRho = eigenvaluesCheck(rback)

	chol_exp = [ vectorization_new(cleanRho)  ]

	try:
		#use eigenvaluesCheck in case the states are not pure ones.

		chol_theoretic = vectorization_new(PureEigenvaluesCheck(rho_startA))
		#hsm.append(HSdist(rho_start0,rback))
		totarr.append( np.concatenate((chol_exp, chol_theoretic), axis=None) )

		#LI metrics
		#totarr.append(fidelity(Qobj(cleanRho),Qobj(rho_startA)))
		#hs.append(HSdist(cleanRho,rho_start0 ))
		i+=1
		#print(np.trace(rho_start0@rho_start0))
	except RuntimeError:
		pass
	
	if i == tot:

		np.save(folder +filename,np.array(totarr))
		break
	else:
		pass
	

## PAULI basis data generation

In [ ]:
import os
from scipy.stats import norm

totarr =[]
hs = []
tot =1
i = 0
li = []

trials = 1000

normTrials = np.sqrt(trials/(local_dim**(d)))

folder = '../multinomialData/'
filename = 'Haar4qubitsTrials'+str(trials)+'PAULI.npy'

if not os.path.isdir(folder):
    os.mkdir(folder)

def HSdist(A,B):
    return np.trace((A-B)@(A-B)).real
  
hsm = []
while True:

	#PURE STATES
	rho_start0 = np.array(random_density_matrix(local_dim**d,rank=1, method = "Hillbert_Schmidt").data)
	#rho_start0 = global_depo_channel(rho_start0,0.1)
	borns =  np.array([ np.trace(rho_start0 @ base).real for base in general_basis])

	#OLD ONE
	singleFreqVariance = norm.rvs(size = local_dim**(2*d))/8
	#
	borns_approx =  [ el1+el2 for el1,el2 in zip(borns,singleFreqVariance) ]  

	#reconstruct the approximation, first stage of linear inversion
	rback = sum([ np.round(reconstruction_basis[i],12)*borns_approx[i] for i in range(4**d) ])
	
	#brute force approximation upon LI
	cleanRho = eigenvaluesCheck(rback)

	chol_exp = [ vectorization_new(cleanRho)  ]

	try:
		#LI test
		li.append(fidelity(Qobj(cleanRho),Qobj(rho_start0)))
		#hs.append(HSdist(cleanRho,rho_start0 ))

		
		chol_theoretic = vectorization_new(PureEigenvaluesCheck(rho_start0))
		totarr.append( np.concatenate((chol_exp, chol_theoretic), axis=None) )
		i+=1
		#print(i)
	except RuntimeError:
		pass
	
	if i == tot:

		#np.save(folder +filename,np.array(totarr))
		break
	else:
		pass
	

In [ ]:
np.mean(li)

## 4) OUT OF DISTRIBUTION

here we generate different preparation of the target OAT state evolution (which consists of 100 time steps).


In [6]:
rm = pd.read_pickle('outofdistribution100.pkl').to_numpy()

type(rm[0][0])

numpy.ndarray

In [71]:
import os
from scipy.stats import norm

totarr =[]
tot = rm.shape[0]
i = 0
li2 = []
#trials = 1000
#normTrials = np.sqrt(trials/(local_dim**(d)))

folder = '../articlePlotPaulibla/'
#filename = 'sample11-100stepsPauli4qubitsTrials'+str(trials)+'OfD.npy'
filename = 'sample20-100stepsSic4qubitsFid85OfD.npy'

if not os.path.isdir(folder):
    os.mkdir(folder)

hsm = []
while True:

	borns =  np.array([ np.trace(rm[i][0] @ base).real for base in general_basis])

	#borns_approx = np.random.multinomial(trials, borns)/trials

	#FOR PAULI
	singleFreqVariance = norm.rvs(size = local_dim**(2*d))/18
	borns_approx =  [ el1+el2 for el1,el2 in zip(borns,singleFreqVariance) ]  

	#reconstruct the approximation
	rback = sum([ np.round(reconstruction_basis[i],12)*borns_approx[i] for i in range(4**d) ])
	#brute force approximation upon LI
	cleanRho = eigenvaluesCheck(rback)

	chol_exp = [ vectorization_new(cleanRho)  ]
	try:
		totarr.append( chol_exp )
		#li2.append(fidelity(Qobj(cleanRho ),Qobj(rm[i][0])))
		i+=1
		#print(np.array(totarr).shape),print(i)
	except RuntimeError:
		pass
	
	if i == tot:

		np.save(folder +filename,np.array(totarr))
		break
	else:
		pass
	
